## Rede Neural Multilayer Perceptron para otimização da função:
### $f(x) = cos(x) * cos(3x)$

In [100]:

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter, LogLocator

import numpy as np
import pandas as pd

import random
from math import cos, tanh
from time import sleep

from ipywidgets import *
import ipywidgets as widgets
import re

from IPython.display import display, HTML

########## entradas dos parametros ##########
style = {'description_width': 'initial'}

print('\n\nTaxa de aprendizagem:')
taxa_aprendizagem = widgets.FloatSlider(
    value=0.01,
    min=0.01,
    max=1,
    step=0.01,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.3f',
)
display(taxa_aprendizagem)

print('\n\nQuantidade de neurônios da camada intermediária:')
qnt_neuronios = widgets.IntSlider(
    value=20,
    min=1,
    max=100,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(qnt_neuronios)

print('\n\nQuantidade de amostras:')
qnt_amostras = widgets.IntSlider(
    value=50,
    min=1,
    max=1000,
    step=1,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
display(qnt_amostras)

print('\n\nCritério de parada:')
parada = widgets.RadioButtons(
    options=['Erro', 'Ciclos'],
    disabled=False, style=style
)
display(parada)

print('\n\nNúmero de ciclos:')
ciclos = widgets.IntSlider(
    value=10,min=1,max=1000,step=1,
    continuous_update=False,
    readout=True, style=style
)
display(ciclos)

print('\n\nErro tolerado:')
erro = widgets.FloatSlider(
    value=0.1,min=0,max=1,step=0.001,
    continuous_update=False,
    readout=True, style=style,
    readout_format='.5f',
)
display(erro)

class RedeMLP():
    def __init__(self, conjunto_entradas, targets, taxa_aprendizagem, parada, ciclos_max, 
                 erro_tolerado, qnt_neuronios, **kwargs):
        self.tamanho_entradas = 256 #tamanho da imagem
        self.numero_saidas = 10 #0-10
        self.conjunto_entradas = np.array(conjunto_entradas)
        self.targets = np.array(targets)
        self.taxa_aprendizagem = taxa_aprendizagem
        self.ciclos = 0
        self.ciclos_max = ciclos_max
        self.parada = parada
        #Quantidade de neuronios seleciondos para camada intermediária
        self.qnt_neuronios_intermediarios = qnt_neuronios
        self.erro_tolerado = erro_tolerado
        #Pesos das ligações entre as camadas inicial e intermediária; intermediária e saída
        self.pesos_ini_inter, self.pesos_inter_saida = self.__inicializa_pesos()
        #Iniciliza os bias da camada intermediária
        self.bias_intermediario = np.array([random.uniform(-0.5,0.5) for saida in range(qnt_neuronios)])
        #Inicializa o bias da camada de saida
        self.bias_saida = np.array(random.uniform(-0.5,0.5))
        self.erro = 0
        
    def __inicializa_pesos(self):
        pesos_ini_inter = []
        pesos_inter_saida = []
        
        #mxn = onde m é o número de entradas e n é o número de neurônios da camada intermediária
        for i in range(self.tamanho_entradas):
            peso_aux = []
            for j in range(self.qnt_neuronios_intermediarios):
                peso_aux.append(random.uniform(-0.5,0.5))
            pesos_ini_inter.append(peso_aux)
        
        #nxk onde n é o número de neurônios da camada intermediária e k é o número de saídas
        for i in range(self.qnt_neuronios_intermediarios):
            peso_aux = []
            for i in range(self.numero_saidas):
                peso_aux.append(random.uniform(-0.5,0.5))
            pesos_inter_saida.append(peso_aux)
        return np.array(pesos_ini_inter), np.array(pesos_inter_saida)
    
    def __f_ativacao_camada(self, zin):
        out = tanh(zin)
        return out
    
    def treinamento(self):
        #print("#Parametros da função")
        treinada = False
        erros = []
        
        while not treinada:
            i = 0 
            self.erro = 0
            
            for entrada_i in self.conjunto_entradas:
                #print("\n\n#################entrada", self.ciclos, "##############")
                ##############Camada inicial para intermediária
                #resultado = 1xn, onde n é o número de neurônios da camada intermediária
                zin = entrada_i @ self.pesos_ini_inter + self.bias_intermediario
                #print("Zin =", zin)
                z = []
                for lista in zin:
                    aux = []
                    for zini in lista:
                        aux.append(self.__f_ativacao_camada(zini))
                    #saída da camada intermediária, entrada da camada de saída
                    z.append(aux)
                    
                z = np.array(z)
                #print("Z =", z)
                
                ##############Camada intermediária para saída
                yin = z @ self.pesos_inter_saida + self.bias_saida
                #print('yin: ', yin[0])
                y = self.__f_ativacao_camada(yin[0])
                
                #Cálculo do erro
                self.erro += 0.5 * np.power((self.targets[i] - y), 2)
                #print('erro:', self.erro)
                
                #########cálculo dos parâmetros para correção dos erros da camada intermediária para camada de saída
                #Calculo dos parâmetros para correção dos pesos e do bias
                deltak = np.multiply((self.targets[i] - y), (1 + y))
                deltak = np.multiply(deltak, (1 - y))
                #print('deltak: ', deltak)
                
                i+=1
                
                #Parâmetro para correção dos pesos
                delta_pesos_inter_saida = np.multiply(self.taxa_aprendizagem, deltak)
                delta_pesos_inter_saida = np.multiply(delta_pesos_inter_saida, z)
                
                #Parâmetro para correção do bias
                delta_bias_saida = np.multiply(self.taxa_aprendizagem, deltak)
                
                #print('delta pesos de saida: ', delta_pesos_inter_saida)
                #print('delta bias de saida: ', delta_bias_saida)
                
                #########cálculo dos parâmetros para correção dos erros da camada entrada para camada de intermediária
                #Calculo dos parâmetros para correção dos pesos e do bias
                delta_in = np.multiply(deltak.T, self.pesos_inter_saida.T)
                
                deltaj = np.multiply(delta_in, (1+z))
                deltaj = np.multiply(deltaj, (1-z))
                #print('delta_in: ', delta_in)
                #print('deltaj: ', deltaj)
                
                #Parâmetro para correção dos pesos
                delta_pesos_ini_inter = np.multiply(self.taxa_aprendizagem, deltaj.T)
                delta_pesos_ini_inter = np.multiply(delta_pesos_ini_inter, entrada_i)
                
                delta_bias_intermediario = np.multiply(self.taxa_aprendizagem, deltaj)
                #print('delta pesos de intermediário: ', delta_pesos_ini_inter)
                #print('delta bias de intermediário: ', delta_bias_intermediario)

                #Correção dos pesos da camada intermediária para camada de saída e bias da camada de saída
                self.pesos_inter_saida = self.pesos_inter_saida + delta_pesos_inter_saida.T
                self.bias_saida = self.bias_saida + delta_bias_saida.T
                #print("Pesos corrigidos da camada saída:", self.pesos_inter_saida)
                #print("bias corrigidos da camada saída:", self.bias_saida)
                
                #Correção dos pesos da camada intermediária para camada de saída e bias da camada de intermediária
                self.pesos_ini_inter = self.pesos_ini_inter + delta_pesos_ini_inter.T
                self.bias_intermediario = self.bias_intermediario + delta_bias_intermediario
                #print("Pesos corrigidos da camada intermediária:", self.pesos_ini_inter)
                #print("bias corrigidos da camada intermediária:", self.bias_intermediario)
            erros.append(self.erro)
            if (self.parada == 'ciclos' and self.ciclos >= self.ciclos_max) \
                or (self.parada == 'erro' and self.erro <= self.erro_tolerado):
                #print('erro final: ', self.erro)
                treinada = True
            else:
                self.ciclos += 1
                
        return list(range(0, self.ciclos+1)), erros
    
    def operacao(self, conjunto_testes):
        saidas_mlp = []
        for entrada_i in conjunto_testes:
            zin = entrada_i * self.pesos_ini_inter + self.bias_intermediario

            z = []
            for lista in zin:
                aux = []
                for zini in lista:
                    aux.append(self.__f_ativacao_camada(zini))

                z.append(aux)
                
            z = np.array(z)

            yin = z @ self.pesos_inter_saida + self.bias_saida
            y = self.__f_ativacao_camada(yin[0])
            saidas_mlp.append(y)
        return saidas_mlp

##### definindo as entradas a partir do arquivo digitostreinamento900.txt #####
def get_inputs_from_txt():
    inputs = []
    file = open('digitos/digitostreinamento900.txt', 'r')
    for line in file :
        line = line.split(" ")
        ipt = []
        for element in line:
            if element != "":
                ipt.append(float(element))
        inputs.append(ipt)
    return inputs
##############################################

def get_targets():
    return pd.read_csv('targets.csv').values

def processar():
    conjunto_entradas = get_inputs_from_txt()
    targets = get_targets()
    #print("Conjunto de entradas:", conjunto_entradas)
    #print("targets: ", targets)
    mlp = RedeMLP(
        conjunto_entradas = conjunto_entradas, 
        targets = targets, 
        taxa_aprendizagem = taxa_aprendizagem.value,
        parada = parada.value.lower(),
        ciclos_max = ciclos.value,
        erro_tolerado = erro.value,
        qnt_neuronios = qnt_neuronios.value
    )
    '''
    print("#########################Parametros iniciais: ")
    print("Entradas: ", mlp.conjunto_entradas)
    print("pesos: ", mlp.pesos_ini_inter)
    print("bias: ", mlp.bias_intermediario)
    print("pesos saida: ", mlp.pesos_inter_saida)
    print("bias saida: ", mlp.bias_saida)
    '''
    #print("Entradas: ", mlp.conjunto_entradas)
    lista_ciclos, lista_erros = mlp.treinamento()
    
    #Operação da MLP para o conjunto de entradas (usado no treinamento)
    aproximacao_mlp = mlp.operacao(conjunto_entradas)
    
    conjunto_testes = gera_conjunto_entrada(-5, 5, 10000)
    targets_testes = gera_targets(conjunto_testes)
    #Operação da MLP para o conjunto de testes (usado no treinamento)
    aproximacao_mlp_testes = mlp.operacao(conjunto_testes)
    
    #print("targets", targets)
    #print("saidas", aproximacao_mlp)
    
    
    ####### plota grafico erro x ciclo#######
    plt.rcParams["figure.figsize"] = [10,6]
    plt.plot(lista_ciclos, lista_erros, 'r', label="Erro", marker='o', scalex=True)
    plt.style.use('ggplot')
    plt.title('Erro x Ciclo')
    plt.xlabel('Ciclo')
    plt.ylabel('Erro')
    plt.legend()
    plt.show()
    #############################
    
    ####### plota o gráfico da função aplicada ao conjunto de treinamento#######
    plt.rcParams["figure.figsize"] = [10,6]
    plt.title('Conjunto de treinamento aplicado a rede')
    plt.plot(conjunto_entradas, targets, 'b', label="Original", marker='o', scalex=True)
    plt.plot(conjunto_entradas, aproximacao_mlp, 'r', label="Aproximação", marker='o', scalex=True)
    plt.style.use('ggplot')
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend()
    plt.show()
    #############################
    
    ####### plota o gráfico da função aplicada a um conjunto de operação#######
    plt.rcParams["figure.figsize"] = [10,6]
    plt.title('Conjunto de operação aplicado a rede')
    plt.plot(conjunto_testes, targets_testes, 'b', label="Original", marker='o', scalex=True)
    plt.plot(conjunto_testes, aproximacao_mlp_testes, 'r', label="Aproximação", marker='o', scalex=True)
    plt.style.use('ggplot')
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.legend()
    plt.show()
    #############################
    

widgets.interact_manual.opts['manual_name'] = 'Treinar a rede' # muda texto do botao
interact_manual(processar); # metodo a executar quando pressionar o botao



Taxa de aprendizagem:


FloatSlider(value=0.01, continuous_update=False, max=1.0, min=0.01, readout_format='.3f', step=0.01)



Quantidade de neurônios da camada intermediária:


IntSlider(value=20, continuous_update=False, min=1)



Quantidade de amostras:


...



Número de ciclos:


IntSlider(value=10, continuous_update=False, max=1000, min=1, style=SliderStyle(description_width='initial'))



Erro tolerado:


FloatSlider(value=0.1, continuous_update=False, max=1.0, readout_format='.5f', step=0.001, style=SliderStyle(d…

interactive(children=(Button(description='Treinar a rede', style=ButtonStyle()), Output()), _dom_classes=('wid…

In [74]:
##### definindo as entradas a partir do arquivo digitostreinamento900.txt #####
def get_inputs_from_txt():
    inputs = []
    file = open('digitos/digitostreinamento900.txt', 'r')
    for line in file :
        line = line.split(" ")
        ipt = []
        for element in line:
            if element != "":
                ipt.append(float(element))
        inputs.append(ipt)
    return inputs
##############################################

In [96]:
inputs = get_inputs_from_txt()
print(len(inputs[1]))


256


In [86]:
gera_targets()

900


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

0
